In [ ]:
!pip install js2py


     |████████████████████████████████| 1.0 MB 5.3 MB/s 
  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-py3-none-any.whl size=26000 sha256=cb3b85bc47485b1418838ce8aa0d06767e13456f68c7b590a73f59d9f4d772d4
  Stored in directory: /root/.cache/pip/wheels/2a/80/ac/dcd2bdbd03dd2b7b7e2bf3e5afbda6a1ab7935bbce314969da
Successfully built pyjsparser
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.3 MB 44.6 MB/s 
     |████████████████████████████████| 16.7 MB 323 kB/s 


In [ ]:
import warnings
warnings.filterwarnings("ignore")

from IPython.display import display, HTML, IFrame
import math
import ast
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import MultiPoint
from shapely import wkt
import js2py
from scipy import stats
import shapely
import matplotlib.pyplot as plt
import bs4, re
import copy
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.request import urlopen
import requests
import json
%matplotlib inline

from pandas.io.json import json_normalize
import requests

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
def getBounds(txt):
  dictionary = {"{":'(', "}":')', " ":'', "[":'', "]":''}
  transTable = txt.maketrans(dictionary)
  txt = txt.translate(transTable)
  txt = txt.replace("'latitude':",'')
  txt = txt.replace("'longitude':",'')

  lst = list(ast.literal_eval(txt))
  points = MultiPoint(lst)
  b = points.bounds

  return b

In [ ]:
def getPoly(txt):
  dictionary = {"{":'(', "}":')', " ":'', "[":'', "]":''}
  transTable = txt.maketrans(dictionary)
  txt = txt.translate(transTable)
  txt = txt.replace("'latitude':",'')
  txt = txt.replace("'longitude':",'')
  txt = txt.replace('"','')
  txt = txt.replace(",",' ')
  txt = txt.replace(") (",',')  

  return txt

In [ ]:
def BusStats(city,data_type):
  #print('Running Bus Stop Density Calculations for ' + city + ' in ' + data_type)
  #import dataframe
  df_location = '/content/gdrive/MyDrive/Columbia/2 Spring 22/ML for Urban Data/ML Final/Project Files/Urban-Mobility-Index/Data/'+ data_type + '/' + city + '/' + city +'.pkl'
  df = pd.read_pickle(df_location)
  df["WKT"] = ""
  df["Count"] = ""
  df["WKTPoly"] = ""  

  i = 0
  for i in range(df.shape[0]):
    df["WKTPoly"][i] = copy.deepcopy(df["Polygon"][i])
    i = i + 1

  #reverse polygon
  i = 0

  for i in range(df.shape[0]):
    for j in range(len(df['Polygon'][i])):
      df['WKTPoly'][i][j] = str(dict(reversed(list(df['WKTPoly'][i][j].items()))))
    if df['WKTPoly'][i][0] != df['WKTPoly'][i][-1]:
      df['WKTPoly'][i].append(df['WKTPoly'][i][0])
    else:
      j = j + 1
  i = i + 1

  #format WKT column as per geopandas
  i = 0
  url = "https://z.overpass-api.de/api/interpreter"

  for i in range(df.shape[0]):
    p = df['WKTPoly'][i]
    df['WKT'][i] = 'POLYGON (' + getPoly(str(p)) + ')'
    #get bus stop nodes for polygon
    query = f"""
      [out:json][timeout:25];
      node["public_transport"="platform"]["bus"="yes"]
      {getBounds(str(df['Polygon'][i]))};
      out skel;
    """
    r = requests.get(url, params={'data': query})
    #handle errors
    try:
      json_normalize(r.json()['elements'])[['id']]
    except:
      df['Count'][i] = 0
    else:
      #Points Geopandas GeoDataFrame   
      bus = json_normalize(r.json()['elements'])[['id']]  # create a DataFrame from the data
      df['Count'][i] = int(bus.shape[0])

  #drop columns we don't need anymore
  df['geometry'] = df.WKT.apply(wkt.loads)
  df.drop(['Bike Score', 'URL','WKT','Polygon' ,'WKTPoly','EnPath'], axis=1, inplace=True) 

  # Polygon Geopandas GeoDataFrame
  eqArea_gdf = gpd.GeoDataFrame(df, geometry='geometry', crs = {'init': 'epsg:4326'}) #wgs84 projection - terrible for area calculations
  eqArea_gdf= eqArea_gdf.to_crs({'init': 'epsg:2163'}) #equal area projection 
  eqArea_gdf["area"] = eqArea_gdf['geometry'].area/ 10**6 #area in sq. km!

  # Calculate Fields
  eqArea_gdf["perSqKm"] = eqArea_gdf["Count"]/eqArea_gdf["area"] #per square kilometer
  eqArea_gdf['Population'] = eqArea_gdf['Population'].str.replace(',', '')
  eqArea_gdf['Population'] = eqArea_gdf['Population'].astype(int)
  eqArea_gdf["perThouCap"] = eqArea_gdf["Count"]*1000/eqArea_gdf["Population"] #per thousand capita

  #Export file
  pkl_out = '/content/gdrive/MyDrive/Columbia/2 Spring 22/ML for Urban Data/ML Final/Project Files/Urban-Mobility-Index/Data/' + data_type +'/'+ city + '/Bus Stops/Density.pkl'
  eqArea_gdf.to_pickle(pkl_out)
  print("Successfully Exported Bus Stop Density of " + city + ' to ' + data_type + '.')

In [ ]:
input_data = {'city':['Ann Arbor', 'Boulder', 'Chicago', 'DC', 'NYC', 'SF','Madison','Tulsa','Seattle'],
        'data_type':['Training','Training','Training','Training','Training','Training','Testing','Testing','Testing']}
input_df = pd.DataFrame(input_data)
input_df

,city,data_type
0,Ann Arbor,Training
1,Boulder,Training
2,Chicago,Training
3,DC,Training
4,NYC,Training
5,SF,Training
6,Madison,Testing
7,Tulsa,Testing
8,Seattle,Testing


In [ ]:
#n = 0

#for n in range(input_df.shape[0]):
#  city = input_df['city'][n]
#  data_type = input_df['data_type'][n]
#  BusStats(city,data_type)
#  n = n + 1